<a href="https://colab.research.google.com/github/calmrocks/master-machine-learning-engineer/blob/main/MLOps/MLPipelineSagemaker.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Automated Machine Learning Pipeline with Amazon SageMaker

## Overview

This notebook demonstrates how to build an automated ML pipeline using Amazon SageMaker. We'll showcase:
- Automated data processing using SageMaker Processing Jobs
- Model training using SageMaker Training Jobs
- Model deployment using SageMaker Endpoints
- Automated monitoring using Model Monitor
- Automated retraining using SageMaker Pipelines

![ML Pipeline](https://github.com/calmrocks/master-machine-learning-engineer/blob/main/MLOps/Diagrams/MLPipeline.png?raw=1)

## Introduction

This notebook demonstrates how to implement an AWS SageMaker ML pipeline in Google Colab. Before we proceed with the pipeline implementation, we need to set up AWS credentials.

### Credential Setup Method
We'll use interactive forms to securely input AWS credentials. This method:
- Keeps credentials temporary (only for current session)
- Avoids storing sensitive information in the notebook
- Uses password fields to hide sensitive input
- Clears the form after credentials are set

### Required AWS Information
You'll need the following information ready:
1. **AWS Access Key ID**: Your AWS account access key
2. **AWS Secret Access Key**: Your AWS account secret key
3. **AWS Region**: The AWS region you want to work in (e.g., 'us-east-1')
4. **S3 Bucket**: The name of your S3 bucket for storing pipeline artifacts
5. **Role ARN**: The Amazon Resource Name of your IAM role with SageMaker permissions

### Prerequisites
Make sure you have:
- An active AWS account
- IAM user with appropriate permissions
- S3 bucket created
- IAM role configured for SageMaker

Run the following cell to set up your credentials:


In [ ]:
!pip install boto3 sagemaker ipywidgets s3fs

## Getting AWS Credentials

There are several ways to obtain and use AWS credentials depending on your setup:

### If Using Amazon SageMaker Notebook Instance

If you're running this notebook in a SageMaker notebook instance, you can leverage the instance's built-in credentials:

```python
import sagemaker
import boto3

# Get the default SageMaker session
sagemaker_session = sagemaker.Session()

# Get the role ARN
role = sagemaker.get_execution_role()

# Get the default bucket
default_bucket = sagemaker_session.default_bucket()

# Get the boto3 session
session = boto3.Session()

# Print details
print(f"Role ARN: {role}")
print(f"Default bucket: {default_bucket}")
```

In [3]:
import os
import boto3
import sagemaker
import logging
from IPython.display import clear_output

access_key = input("AWS Access Key ID: ")
secret_key = input("AWS Secret Access Key: ")
session_token = input("AWS Session Token (press Enter if none): ").strip() or None
region = input("AWS Region (default: us-east-1): ") or "us-east-1"
bucket = input("S3 Bucket Name: ")
role_arn = input("Role ARN: ")

print("\nCredentials set:")
print(f"Access Key: {access_key[:4]}...{access_key[-4:]}")
print(f"Secret Key: {secret_key[:4]}...{secret_key[-4:]}")
if session_token:
    print(f"Session Token: {session_token[:4]}...{session_token[-4:]}")
print(f"Region: {region}")
print(f"Bucket: {bucket}")
print(f"Role ARN: {role_arn}\n")

session = boto3.Session(
    aws_access_key_id=access_key,
    aws_secret_access_key=secret_key,
    aws_session_token=session_token,
    region_name=region
)

clear_output()

s3 = session.client('s3')
bucket_name = bucket
try:
    s3.head_bucket(Bucket=bucket_name)
    print(f"✓ Successfully accessed S3 bucket: {bucket_name}")
except Exception as e:
    print(f"❌ Error accessing S3 bucket: {str(e)}")


✓ Successfully accessed S3 bucket: yinglonw-test-us-east-1


## Setup Project Files

Before we start building our ML pipeline, we need to get our project files from GitHub. These files contain:

- `pipeline.py`: Main pipeline configuration that orchestrates the ML workflow
- `scripts/preprocessing.py`: Data preprocessing logic including cleaning and feature engineering
- `scripts/training.py`: Model training code using Random Forest algorithm
- `scripts/evaluation.py`: Model evaluation metrics calculation

The code below will:
1. Download these files from GitHub repository
2. Create necessary directories in our notebook environment
3. Save the files locally so we can use them in our pipeline

This setup ensures we have all required scripts available in our SageMaker notebook instance to build and execute our ML pipeline.

In [66]:
import requests
import os

def download_github_file(github_url, local_path):
    """
    Download a file from GitHub and save it locally.
    Converts GitHub web URL to raw content URL.
    """
    # Convert GitHub URL to raw content URL
    raw_url = github_url.replace('github.com', 'raw.githubusercontent.com').replace('/blob/', '/')

    # Create directory if it doesn't exist
    os.makedirs(os.path.dirname(local_path), exist_ok=True)

    # Download and save the file
    response = requests.get(raw_url)
    if response.status_code == 200:
        with open(local_path, 'w') as f:
            f.write(response.text)
        print(f"Successfully downloaded: {local_path}")
    else:
        print(f"Failed to download: {local_path}")
        print(f"Status code: {response.status_code}")

# Define the files to download
files = {
    'pipeline.py': 'https://github.com/calmrocks/master-machine-learning-engineer/blob/main/MLOps/wine_quality_pipeline/pipeline.py',
    'scripts/preprocessing.py': 'https://github.com/calmrocks/master-machine-learning-engineer/blob/main/MLOps/wine_quality_pipeline/scripts/preprocessing.py',
    'scripts/training.py': 'https://github.com/calmrocks/master-machine-learning-engineer/blob/main/MLOps/wine_quality_pipeline/scripts/training.py',
    'scripts/evaluation.py': 'https://github.com/calmrocks/master-machine-learning-engineer/blob/main/MLOps/wine_quality_pipeline/scripts/evaluation.py'
}

# Download all files
for local_path, github_url in files.items():
    download_github_file(github_url, f'wine_quality_pipeline/{local_path}')

print("\nChecking downloaded files:")
print(list(Path("wine_quality_pipeline").glob("**/*.py")))

Successfully downloaded: wine_quality_pipeline/pipeline.py
Successfully downloaded: wine_quality_pipeline/scripts/preprocessing.py
Successfully downloaded: wine_quality_pipeline/scripts/training.py
Successfully downloaded: wine_quality_pipeline/scripts/evaluation.py

Checking downloaded files:
[PosixPath('wine_quality_pipeline/pipeline.py'), PosixPath('wine_quality_pipeline/scripts/evaluation.py'), PosixPath('wine_quality_pipeline/scripts/preprocessing.py'), PosixPath('wine_quality_pipeline/scripts/training.py')]


# Wine Quality ML Pipeline with Amazon SageMaker

This notebook demonstrates how to build an end-to-end machine learning pipeline using Amazon SageMaker. We'll use the Wine Quality dataset to showcase:
- Data preprocessing
- Model training
- Model evaluation
- Automated retraining
- Model monitoring

The pipeline will automatically handle data preprocessing, model training, and evaluation, making it easy to retrain models when new data arrives.

## Setup and Import Required Libraries

First, let's import our required libraries and setup our project structure.

In [69]:
import boto3
import sagemaker
import pandas as pd
from datetime import datetime
from pathlib import Path

# Import our pipeline creation function
from wine_quality_pipeline.pipeline import create_pipeline

print("Current working directory:", Path.cwd())
print("\nContents of wine_quality_pipeline:")
print(list(Path("wine_quality_pipeline").glob("**/*.py")))

Current working directory: /content

Contents of wine_quality_pipeline:
[PosixPath('wine_quality_pipeline/pipeline.py'), PosixPath('wine_quality_pipeline/scripts/evaluation.py'), PosixPath('wine_quality_pipeline/scripts/preprocessing.py'), PosixPath('wine_quality_pipeline/scripts/training.py')]


## Download and Prepare Initial Dataset

First, let's download the Wine Quality dataset and upload it to our S3 bucket. We'll use this as our initial training data.

In [61]:
import pandas as pd
from datetime import datetime
import io
from urllib.parse import urlparse

# Download wine quality dataset
wine_data = pd.read_csv(
    'https://archive.ics.uci.edu/ml/machine-learning-databases/wine-quality/winequality-red.csv',
    sep=';'
)

# Define S3 path
initial_data_path = f"s3://{bucket}/wine-quality/raw/winequality.csv"

# Convert DataFrame to CSV buffer
csv_buffer = io.StringIO()
wine_data.to_csv(csv_buffer, index=False)

# Parse S3 URL to get bucket and key
def parse_s3_url(s3_url):
    parsed = urlparse(s3_url)
    bucket = parsed.netloc
    key = parsed.path.lstrip('/')
    return bucket, key

bucket_name, key_path = parse_s3_url(initial_data_path)

# Upload using the session's S3 client
s3 = session.client('s3')
s3.put_object(
    Bucket=bucket_name,
    Key=key_path,
    Body=csv_buffer.getvalue()
)

print(f"Data uploaded to: {initial_data_path}")
print(f"Dataset shape: {wine_data.shape}")
print("\nFeatures:")
print(wine_data.columns.tolist())

Data uploaded to: s3://yinglonw-test-us-east-1/wine-quality/raw/winequality.csv
Dataset shape: (1599, 12)

Features:
['fixed acidity', 'volatile acidity', 'citric acid', 'residual sugar', 'chlorides', 'free sulfur dioxide', 'total sulfur dioxide', 'density', 'pH', 'sulphates', 'alcohol', 'quality']


## Upload Scripts to S3

Now, let's upload our existing preprocessing, training, and evaluation scripts to S3.

In [70]:
def upload_scripts_to_s3(boto3_session, bucket, prefix="wine-quality/code"):
    # Get S3 client from boto3 session
    s3_client = boto3_session.client('s3')

    # Define the scripts to upload
    scripts = {
        'preprocessing.py': 'wine_quality_pipeline/scripts/preprocessing.py',
        'training.py': 'wine_quality_pipeline/scripts/training.py',
        'evaluation.py': 'wine_quality_pipeline/scripts/evaluation.py'
    }

    # Upload each script
    for s3_key, local_path in scripts.items():
        try:
            s3_client.upload_file(
                local_path,
                bucket,
                f"{prefix}/{s3_key}"
            )
            print(f"Uploaded {local_path} to s3://{bucket}/{prefix}/{s3_key}")
        except Exception as e:
            print(f"Error uploading {local_path}: {str(e)}")

    return f"s3://{bucket}/{prefix}"

# Upload scripts using the boto3 session
script_prefix = upload_scripts_to_s3(session, bucket)
print("\nAll scripts uploaded successfully!")

Uploaded wine_quality_pipeline/scripts/preprocessing.py to s3://yinglonw-test-us-east-1/wine-quality/code/preprocessing.py
Uploaded wine_quality_pipeline/scripts/training.py to s3://yinglonw-test-us-east-1/wine-quality/code/training.py
Uploaded wine_quality_pipeline/scripts/evaluation.py to s3://yinglonw-test-us-east-1/wine-quality/code/evaluation.py

All scripts uploaded successfully!


## Create and Configure Pipeline

Now we'll create our SageMaker pipeline using our existing pipeline configuration.

In [74]:
import importlib
import wine_quality_pipeline.pipeline
importlib.reload(wine_quality_pipeline.pipeline)
from wine_quality_pipeline.pipeline import create_pipeline

In [75]:
# Create pipeline
pipeline, model_monitor = create_pipeline(
    role=role_arn,
    bucket=bucket,
    pipeline_name="WineQualityPipeline",
    base_job_prefix="wine-quality",
    boto3_session=session
)

print("Pipeline created successfully!")

Pipeline created successfully!


## Set Up Model Monitoring

Configure monitoring for our model to track its performance over time.

In [55]:
def setup_cloudwatch_alerts(cloudwatch_client):
    """Setup CloudWatch alerts for model monitoring"""
    try:
        # Example alert for model performance degradation
        cloudwatch_client.put_metric_alarm(
            AlarmName='WineQualityModelDegradation',
            MetricName='mse',
            Namespace='WineQualityModel',
            Statistic='Average',
            Period=300,
            EvaluationPeriods=2,
            Threshold=0.5,
            ComparisonOperator='GreaterThanThreshold'
            # Add AlarmActions to get notified
        )
        print("CloudWatch alert created successfully!")
    except Exception as e:
        print(f"Error creating CloudWatch alert: {str(e)}")

# Setup CloudWatch alerts
cloudwatch = session.client('cloudwatch')
setup_cloudwatch_alerts(cloudwatch)

print("Model monitoring setup complete!")

CloudWatch alert created successfully!
Model monitoring setup complete!


## Execute Pipeline

Finally, let's execute our pipeline and start monitoring.

In [62]:
# Execute the pipeline
pipeline.upsert(role_arn=role_arn)
execution = pipeline.start()

print(f"Pipeline execution started with ARN: {execution.arn}")

Pipeline execution started with ARN: arn:aws:sagemaker:us-east-1:191514433341:pipeline/WineQualityPipeline/execution/dmw6rirzcmmn


## Monitor Pipeline Execution

Let's check the status of our pipeline execution.

In [65]:
def check_pipeline_status(execution):
    """Monitor the pipeline execution status"""
    print(f"Pipeline execution status: {execution.describe()['PipelineExecutionStatus']}")
    print("\nStep statuses:")
    for step in execution.list_steps():
        print(f"- {step['StepName']}: {step['StepStatus']}")

# Check status
check_pipeline_status(execution)

Pipeline execution status: Executing

Step statuses:
- PreprocessWineData: Executing


## Retrain

To simulate new data arrival and trigger retraining:

In [ ]:
def simulate_new_data():
    """Simulate new data arrival and trigger retraining"""
    # Create modified dataset
    new_data = wine_data.copy()
    new_data['quality'] = new_data['quality'] * 1.1  # Simulate data drift

    # Upload to S3 with new timestamp
    new_timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
    new_data_path = f"s3://{bucket}/wine-quality/data/{new_timestamp}/winequality.csv"
    new_data.to_csv(new_data_path, index=False)

    # Start new pipeline execution
    execution = pipeline.start(
        parameters={
            'InputDataPath': new_data_path,
            'Timestamp': new_timestamp
        }
    )

    return execution

# Uncomment to simulate new data and trigger retraining
# new_execution = simulate_new_data()

# Wine Quality ML Pipeline

![Wine Quality Pipeline Diagram](https://github.com/calmrocks/master-machine-learning-engineer/blob/main/MLOps/Diagrams/WineQualityPipeline.png?raw=1)

## Current Pipeline

The current ML pipeline for the Wine Quality project consists of four main steps:

1. **Preprocessing**: Prepares the raw wine quality data for model training.
2. **Training**: Trains the machine learning model using the preprocessed data.
3. **Evaluation**: Assesses the performance of the trained model.
4. **Condition**: Determines if the model meets the required quality threshold.

## Potential Enhancements

To further improve the pipeline, consider adding the following steps:

1. **Model Registration**: Register the approved model in the SageMaker Model Registry.
2. **Deployment**: Automate the deployment of successful models to a SageMaker endpoint.
3. **Hyperparameter Tuning**: Implement automated hyperparameter optimization.
4. **A/B Testing**: Set up a mechanism for comparing new models against the current production model.
5. **Monitoring**: Add continuous monitoring of the deployed model for data drift and performance degradation.
6. **Automated Retraining**: Implement a feedback loop to trigger retraining based on monitoring results.
7. **Feature Store**: Integrate with SageMaker Feature Store for better feature management.
8. **Multi-Model Deployment**: Explore deploying multiple models for different use cases or customer segments.

These additions would create a more comprehensive MLOps pipeline, enabling continuous integration, deployment, and improvement of the Wine Quality prediction model.

# Next Steps

After successfully creating and running the ML pipeline, consider the following advanced steps to enhance your ML workflow:

## 1. Model Registry and Version Management
- Implement SageMaker Model Registry to catalog and version your models
- Set up automated model approval workflows
- Create model groups for organizing related models

## 2. Inference Optimization
- Explore different inference types:
  - Real-time inference with SageMaker Endpoints
  - Batch transform for large-scale inference
  - Serverless Inference for cost-effective, auto-scaling deployments
- Implement multi-model endpoints for serving multiple models efficiently

## 3. Model Monitoring and Maintenance
- Set up data drift detection using SageMaker Model Monitor
- Implement automated retraining pipelines based on performance metrics
- Establish A/B testing framework for model comparisons

## 4. Multi-Region Setup
- Deploy models across multiple AWS regions for improved availability and reduced latency
- Implement cross-region model replication strategies
- Set up global endpoints with Amazon SageMaker Edge Manager

## 5. MLOps and CI/CD
- Integrate the ML pipeline with CI/CD tools (e.g., AWS CodePipeline, GitHub Actions)
- Implement automated testing for data quality, model performance, and infrastructure
- Set up approval gates for model deployment to production

## 6. Cost Optimization
- Analyze and optimize resource utilization (instance types, autoscaling)
- Implement SageMaker Savings Plans for long-term cost reduction
- Explore SageMaker Managed Spot Training for reduced training costs

## 7. Security and Compliance
- Implement fine-grained IAM roles and permissions
- Set up VPC configurations for enhanced network security
- Ensure compliance with relevant regulations (e.g., GDPR, HIPAA) in data handling and model deployment

## 8. Advanced Model Techniques
- Experiment with ensemble methods and model stacking
- Implement automated hyperparameter tuning using SageMaker Hyperparameter Tuning
- Explore SageMaker Feature Store for feature management and sharing

## 9. Explainability and Fairness
- Implement SageMaker Clarify for model explainability and bias detection
- Develop custom explainability dashboards for stakeholders
- Establish regular audits for model fairness and ethical considerations

## 10. Integration with Business Processes
- Develop APIs or interfaces for business users to interact with the model
- Create visualization dashboards for model performance and business impact
- Establish feedback loops for continuous model improvement based on business outcomes

By addressing these areas, you'll create a more robust, scalable, and production-ready ML system that can deliver sustained value to your organization.

## Cleanup (Optional)

If you want to clean up resources:
1. Stop any running pipeline executions
2. Delete the CloudWatch alarms
3. Delete the model monitor
4. Delete the pipeline

Note: Keep these resources if you plan to continue development or monitoring.

In [ ]:
def cleanup_resources(pipeline_name="WineQualityPipeline"):
    """Clean up created resources"""
    try:
        # Delete CloudWatch alarm
        cloudwatch.delete_alarms(AlarmNames=['WineQualityModelDegradation'])
        print("CloudWatch alarm deleted")

        # Delete pipeline
        sagemaker_client = session.client('sagemaker')
        sagemaker_client.delete_pipeline(PipelineName=pipeline_name)
        print("Pipeline deleted")

        print("Cleanup completed successfully!")
    except Exception as e:
        print(f"Error during cleanup: {str(e)}")

# Uncomment to cleanup resources
# cleanup_resources()